In [16]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# Load the preprocessed data
modelling_table = pd.read_csv("preprocessed_data.csv")

# valid_datetimeをdatetime型に変換
modelling_table['valid_datetime'] = pd.to_datetime(modelling_table['valid_datetime'])

# 季節ごとのデータフレームを作成
modelling_table['month'] = modelling_table['valid_datetime'].dt.month
spring = modelling_table[modelling_table['month'].isin([3, 4, 5])]
summer = modelling_table[modelling_table['month'].isin([6, 7, 8])]
autumn = modelling_table[modelling_table['month'].isin([9, 10, 11])]
winter = modelling_table[modelling_table['month'].isin([12, 1, 2])]

# 季節ごとの総発電量モデル
total_models = {
    'spring': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=spring),
    'summer': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=summer),
    'autumn': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=autumn),
    'winter': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=winter)
}

forecast_models = dict()

# 季節ごとの太陽光発電と風力発電の分位数を計算
for quantile in range(10, 100, 10):
    for season, season_months in {'spring': [3, 4, 5], 'summer': [6, 7, 8], 'autumn': [9, 10, 11], 'winter': [12, 1, 2]}.items():
        forecast_models[f"{season}_q{quantile}"] = total_models[season].fit(q=quantile/100, max_iter=2500)
        modelling_table.loc[modelling_table['month'].isin(season_months), f"{season}_q{quantile}"] = forecast_models[f"{season}_q{quantile}"].predict(modelling_table.loc[modelling_table['month'].isin(season_months)])

# 各季節ごとの予測結果を保存
modelling_table.to_csv("analyzed_data_sea.csv", index=False)

seasons = ['spring', 'summer', 'autumn', 'winter']
# 全ての量子回帰モデルを"models_sea"ファイルに保存
for season in seasons:
    for quantile in range(10, 100, 10):
        forecast_models[f"{season}_q{quantile}"].save(f"models_sea/model_{season}_q{quantile}.pickle")

/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
